In [2]:
import tensorflow as tf
print(tf.__version__)

2.5.0-dev20201130


In [3]:
from tensorflow.keras.models import load_model, Model
import matplotlib.pyplot as plt
import cv2

In [4]:
# Read original model
model = load_model('../model_zoo/variouse/issue_00003/fiop_dumb_model.h5')

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Functional)         (None, 3, 3, 2048)        5347520   
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                221196    
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 5,568,829
Trainable params: 5,540,797
Non-trainable params: 28,032
_________________________________________________________________


In [7]:
# Select inner model
inner_model = model.get_layer('model_2')

In [8]:
# Adjust config of the new model
model_config = model.get_config()
fixed_config = inner_model.get_config()

i_layers = fixed_config['input_layers']
o_layers = [[model_config['layers'][-1]['config']['name'], 0, 0]]
# o_layers = [[['flatten_1',0,0]]]

add_layers = []
in_layer = fixed_config['layers'][-1]['name']
for item in model_config['layers'][2:]:
    item['name'] = item['config']['name']
    item['inbound_nodes'] = [[[in_layer, 0, 0, {}]]]
    in_layer = item['config']['name']
    add_layers.append(item)
    
fixed_config['layers'] += add_layers
fixed_config['input_layers'] = i_layers
fixed_config['output_layers'] = o_layers

fixed_model = Model.from_config(fixed_config)

In [10]:
# Transfer weights to the new model
for layer in inner_model.layers:
    fixed_model.get_layer(layer.name).set_weights(layer.get_weights())
    
for layer in model.layers[2:]:
    fixed_model.get_layer(layer.name).set_weights(layer.get_weights())

In [11]:
in_data = cv2.imread('../unit_test/unit_test_data/person_640x640.jpg')
in_data = cv2.resize(in_data, tuple(reversed(model.input_shape[1:3])))[None,...]

In [12]:
# Check the correctnes of the predictions
print(model.predict(in_data))
print(fixed_model.predict(in_data))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[[0.20348462]]
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When fili

In [13]:
fixed_model.save('../model_zoo/variouse/issue_00003/fiop_dumb_model_fixed.h5')